In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,Mission Outcome,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,Success,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,Success,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,Success,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,Success,0,677.0,F9 v1.0 B0007,v1.0


In [3]:
# Prepare site list
site_list = ['All','CCAFS LC-40','CCAFS SLC-40','KSC LC-39A','VAFB SLC-4E']
site_list

['All', 'CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E']

In [4]:
# Create a dash application
app = JupyterDash(__name__)

In [5]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(
                                    id='site-dropdown',
                                    options = [{'label': i , 'value':i} for i in site_list],
                                    value = 'All',
                                    placeholder = 'Select a Launch Site here',
                                    searchable =True),

                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site                    
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(
                                    id='payload-slider',
                                    min =0, 
                                    max = 10000, 
                                    step = 1000, 
                                    marks = {0:'0', 2500: '2500', 5000:'5000', 7500:'7500', 10000:'10000'},
                                    value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                            ])

In [6]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id = "success-pie-chart", component_property = 'figure'), 
            Input(component_id = 'site-dropdown', component_property= 'value'))
def get_pie_chart (entered_site):
    if entered_site == 'All':        
        filtered_df = spacex_df[spacex_df['class']==1].groupby('Launch Site').count().reset_index()
        fig = px.pie(filtered_df, 
        values = 'class',
        names='Launch Site',
        title = 'Total Success Launches By Site')
        return fig
    
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site].groupby('class').count().reset_index()
        fig = px.pie(filtered_df, 
        values = 'Launch Site',
        names='class',
        title = 'Launch Outcome Chart for Site '+ entered_site)
        return fig

In [7]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id = "success-payload-scatter-chart", component_property = 'figure'), 
              [Input(component_id = 'site-dropdown', component_property= 'value'),
              Input(component_id = 'payload-slider', component_property= 'value')])

def get_scatter_chart (entered_site, slider_range):
    min, max = slider_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >min)&(spacex_df['Payload Mass (kg)'] <max)]
    if entered_site == 'All':
        fig = px.scatter(filtered_df,
        x='Payload Mass (kg)', 
        y = 'class',
        title = 'Correlaton Between Payload and Success for All Sites',
        color = 'Booster Version Category')
        return fig

    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, 
        x='Payload Mass (kg)', 
        y = 'class',
        title = 'Correlaton Between Payload and Success for Site '+ entered_site,
        color = 'Booster Version Category')
        return fig

In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)


run the dashboard at http://localhost:8090/